Load Data into a DataFrame

In [0]:
# Load the synthetic data into a DataFrame
data_file_path = "/FileStore/tables/Salary_Data.csv"
df = spark.read.csv(data_file_path, header=True, inferSchema=True)

In [0]:
# Display schema of DataFrame
df.printSchema()

# Show the initial DataFrame
print("Initial DataFrame:")
df.show(10)

root
 |-- Age: integer (nullable = true)
 |-- Gender: string (nullable = true)
 |-- Education Level: string (nullable = true)
 |-- Job Title: string (nullable = true)
 |-- Years of Experience: double (nullable = true)
 |-- Salary: integer (nullable = true)

Initial DataFrame:
+---+------+---------------+--------------------+-------------------+------+
|Age|Gender|Education Level|           Job Title|Years of Experience|Salary|
+---+------+---------------+--------------------+-------------------+------+
| 32|  Male|     Bachelor's|   Software Engineer|                5.0| 90000|
| 28|Female|       Master's|        Data Analyst|                3.0| 65000|
| 45|  Male|            PhD|      Senior Manager|               15.0|150000|
| 36|Female|     Bachelor's|     Sales Associate|                7.0| 60000|
| 52|  Male|       Master's|            Director|               20.0|200000|
| 29|  Male|     Bachelor's|   Marketing Analyst|                2.0| 55000|
| 42|Female|       Master's|  

Register the DataFrame as a Temporary Table

In [0]:
# Register the DataFrame as a Temporary Table
df.createOrReplaceTempView("my_table")

Perform SQL-like Queries

In [0]:
# Select all rows where age is greater than 25
result = spark.sql("SELECT * FROM my_table WHERE Age > 25")

result.show()

+---+------+---------------+--------------------+-------------------+------+
|Age|Gender|Education Level|           Job Title|Years of Experience|Salary|
+---+------+---------------+--------------------+-------------------+------+
| 32|  Male|     Bachelor's|   Software Engineer|                5.0| 90000|
| 28|Female|       Master's|        Data Analyst|                3.0| 65000|
| 45|  Male|            PhD|      Senior Manager|               15.0|150000|
| 36|Female|     Bachelor's|     Sales Associate|                7.0| 60000|
| 52|  Male|       Master's|            Director|               20.0|200000|
| 29|  Male|     Bachelor's|   Marketing Analyst|                2.0| 55000|
| 42|Female|       Master's|     Product Manager|               12.0|120000|
| 31|  Male|     Bachelor's|       Sales Manager|                4.0| 80000|
| 26|Female|     Bachelor's|Marketing Coordin...|                1.0| 45000|
| 38|  Male|            PhD|    Senior Scientist|               10.0|110000|

In [0]:
# Compute the average salary by gender
avg_salary_by_gender = spark.sql("SELECT Gender, AVG(Salary) as avg_salary FROM my_table GROUP BY Gender")
avg_salary_by_gender.show()

+------+------------------+
|Gender|        avg_salary|
+------+------------------+
|  null|              null|
|Female| 97011.17318435754|
|  Male|103867.78350515464|
+------+------------------+



Creating and managing temporary views.

In [0]:
# Create a temporary view
df.createOrReplaceTempView("people")

In [0]:
# Query the temporary view
result = spark.sql("SELECT * FROM people WHERE age > 25")

result.show()

+---+------+---------------+--------------------+-------------------+------+
|Age|Gender|Education Level|           Job Title|Years of Experience|Salary|
+---+------+---------------+--------------------+-------------------+------+
| 32|  Male|     Bachelor's|   Software Engineer|                5.0| 90000|
| 28|Female|       Master's|        Data Analyst|                3.0| 65000|
| 45|  Male|            PhD|      Senior Manager|               15.0|150000|
| 36|Female|     Bachelor's|     Sales Associate|                7.0| 60000|
| 52|  Male|       Master's|            Director|               20.0|200000|
| 29|  Male|     Bachelor's|   Marketing Analyst|                2.0| 55000|
| 42|Female|       Master's|     Product Manager|               12.0|120000|
| 31|  Male|     Bachelor's|       Sales Manager|                4.0| 80000|
| 26|Female|     Bachelor's|Marketing Coordin...|                1.0| 45000|
| 38|  Male|            PhD|    Senior Scientist|               10.0|110000|

In [0]:
# Check if a temporary view exists
view_exists = spark.catalog.tableExists("people")
view_exists

Out[8]: True

In [0]:
# Drop a temporary view
spark.catalog.dropTempView("people")

Out[9]: True

In [0]:
# Check if a temporary view exists
view_exists = spark.catalog.tableExists("people")
view_exists

Out[10]: False

Subqueries

In [0]:
# Create DataFrames
employee_data = [
    (1, "John"), (2, "Alice"), (3, "Bob"), (4, "Emily"),
    (5, "David"), (6, "Sarah"), (7, "Michael"), (8, "Lisa"),
    (9, "William")
]
employees = spark.createDataFrame(employee_data, ["id", "name"])

salary_data = [
    ("HR", 1, 60000), ("HR", 2, 55000), ("HR", 3, 58000),
    ("IT", 4, 70000), ("IT", 5, 72000), ("IT", 6, 68000),
    ("Sales", 7, 75000), ("Sales", 8, 78000), ("Sales", 9, 77000)
]
salaries = spark.createDataFrame(salary_data, ["department", "id", "salary"])

employees.show()

salaries.show()

+---+-------+
| id|   name|
+---+-------+
|  1|   John|
|  2|  Alice|
|  3|    Bob|
|  4|  Emily|
|  5|  David|
|  6|  Sarah|
|  7|Michael|
|  8|   Lisa|
|  9|William|
+---+-------+

+----------+---+------+
|department| id|salary|
+----------+---+------+
|        HR|  1| 60000|
|        HR|  2| 55000|
|        HR|  3| 58000|
|        IT|  4| 70000|
|        IT|  5| 72000|
|        IT|  6| 68000|
|     Sales|  7| 75000|
|     Sales|  8| 78000|
|     Sales|  9| 77000|
+----------+---+------+



In [0]:
# Register as temporary views
employees.createOrReplaceTempView("employees")
salaries.createOrReplaceTempView("salaries")

In [0]:
# Subquery to find employees with salaries above average
result = spark.sql("""
    SELECT name
    FROM employees
    WHERE id IN (
        SELECT id
        FROM salaries
        WHERE salary > (SELECT AVG(salary) FROM salaries)
    )
""")

result.show()

+-------+
|   name|
+-------+
|  Emily|
|  David|
|Michael|
|   Lisa|
|William|
+-------+



Window Function

In [0]:
from pyspark.sql.window import Window
from pyspark.sql import functions as F

In [0]:
employee_salary = spark.sql("""
    select  salaries.*, employees.name
    from salaries 
    left join employees on salaries.id = employees.id
""")

employee_salary.show()

+----------+---+------+-------+
|department| id|salary|   name|
+----------+---+------+-------+
|        HR|  1| 60000|   John|
|        HR|  2| 55000|  Alice|
|        HR|  3| 58000|    Bob|
|        IT|  4| 70000|  Emily|
|        IT|  5| 72000|  David|
|        IT|  6| 68000|  Sarah|
|     Sales|  7| 75000|Michael|
|     Sales|  8| 78000|   Lisa|
|     Sales|  9| 77000|William|
+----------+---+------+-------+



In [0]:
# Create a window specification
window_spec = Window.partitionBy("department").orderBy(F.desc("salary"))

In [0]:
# Calculate the rank of employees within each department based on salary
employee_salary.withColumn("rank", F.rank().over(window_spec)).show()

+----------+---+------+-------+----+
|department| id|salary|   name|rank|
+----------+---+------+-------+----+
|        HR|  1| 60000|   John|   1|
|        HR|  3| 58000|    Bob|   2|
|        HR|  2| 55000|  Alice|   3|
|        IT|  5| 72000|  David|   1|
|        IT|  4| 70000|  Emily|   2|
|        IT|  6| 68000|  Sarah|   3|
|     Sales|  8| 78000|   Lisa|   1|
|     Sales|  9| 77000|William|   2|
|     Sales|  7| 75000|Michael|   3|
+----------+---+------+-------+----+

